# Testing with Gensim, can skip to next section for final model

In [4]:
import gensim
from gensim.models import LdaMulticore, CoherenceModel, LdaModel, TfidfModel
from gensim.corpora import Dictionary

from joblib import load


/root/miniconda3/envs/nlp/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [ ]:
vec=load('Data/1gram-vectorizer.joblib')
vecs = vec.transform(data.tokes)

In [93]:
data.tokes[0:2]

0    [@ronnie_haydon, almost, certainly, mange, ., ...
1    [[, ivermectin, paste, dewormer, -, 6.08, g, d...
Name: tokes, dtype: object

In [12]:
dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
corpus = [dct.doc2bow(line) for line in data.no_punc] 

In [13]:
for doc in corpus[0:2]:
   print([[dct[id], freq] for id, freq in doc])

[['@ronnie_haydon', 1], ['almost', 1], ['and', 1], ['are', 1], ['at', 1], ['certainly', 1], ['contact', 1], ['do', 1], ['eat', 1], ['for', 2], ['free', 1], ['give', 1], ['happy', 1], ['her', 2], ['i', 2], ['if', 1], ['in', 1], ['it', 2], ['ivermectin', 1], ['last', 1], ['leave', 1], ['local', 1], ['mange', 1], ['most', 1], ['put', 1], ['roughly', 1], ['same', 1], ['saw', 1], ['see', 1], ['some', 1], ['something', 1], ['tasty', 1], ['the', 1], ['then', 1], ['they', 1], ['this', 1], ['time', 1], ['to', 4], ['treat', 1], ['vet', 1], ['vets', 1], ['where', 1], ['will', 1], ['would', 2], ['you', 2], ['your', 1]]
[['ivermectin', 2], ['%', 2], ['-', 4], ['187', 2], ['3', 2], ['608', 2], ['@', 2], ['[', 1], [']', 1], ['apple', 2], ['been', 1], ['dewormer', 2], ['dose', 2], ['flavor', 2], ['g', 2], ['has', 1], ['https//tco/3aj1qfs4hn', 1], ['https//tco/rrloi8hw3n', 1], ['on', 1], ['pack', 2], ['paste', 2], ['pet', 1], ['posted', 1], ['supplies', 1], ['–', 1]]


In [20]:
data[data.no_punc == '']

,datetime,lang,rts,likes,quotes,replies,hr,date,tokes,no_punc


In [36]:
dct.filter_extremes(no_below=10, no_above=0.8, keep_n=100000)
new_corpus = [dct.doc2bow(doc) for doc in data.no_punc]

In [37]:
model = TfidfModel(new_corpus)

In [38]:
tf_corp = model[new_corpus]

In [40]:
lda = LdaMulticore(tf_corp, id2word=dct, num_topics=30, workers=9, iterations=6, eta='auto')
lda.save('1gram-t30-lda.model')

In [41]:
lda.print_topics()

[(8,
  '0.012*"truth" + 0.011*"pfizer" + 0.008*"lives" + 0.007*"saves" + 0.006*"tennessee" + 0.006*"you" + 0.006*"and" + 0.006*"to" + 0.006*"trending" + 0.006*"is"'),
 (17,
  '0.011*"hospital" + 0.010*"|" + 0.010*"-" + 0.010*"patient" + 0.009*"covid" + 0.008*"19" + 0.008*"to" + 0.007*"judge" + 0.006*"a" + 0.006*"wife"'),
 (18,
  '0.014*"zinc" + 0.012*"d" + 0.011*"and" + 0.011*"vitamin" + 0.011*"c" + 0.007*"i" + 0.007*"you" + 0.006*"quercetin" + 0.006*"hcq" + 0.006*"with"'),
 (22,
  '0.018*"visit" + 0.012*"buy" + 0.012*"easily" + 0.012*"-" + 0.011*"anyone" + 0.010*"here" + 0.009*"can" + 0.008*"trial" + 0.007*"study" + 0.007*"studies"'),
 (29,
  '0.017*"😂" + 0.007*"you" + 0.007*"’" + 0.006*"lie" + 0.006*"and" + 0.006*"horse" + 0.006*"to" + 0.006*"that" + 0.006*"i" + 0.006*"""'),
 (9,
  '0.010*"@mysterysolvent" + 0.008*""" + 0.006*"is" + 0.006*"of" + 0.006*"-" + 0.006*"covid" + 0.005*"and" + 0.005*"you" + 0.005*"to" + 0.005*"de-wormer"'),
 (20,
  '0.015*"congress" + 0.010*"members" + 0.00

# Grid Searching Topics

In [1]:
import optuna
import gensim
import pandas as pd
from gensim.models import LdaMulticore, CoherenceModel, LdaModel, TfidfModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
from joblib import load, dump

Given Gensim's built in Coherence Model, I ultimately decided to do dictionary creation, vectorization, LDA modeling all within Gensim

In [2]:
data=pd.read_csv('Data/no_punc_tokes.csv', index_col='Unnamed: 0')
data['no_punc'] = data['no_punc'].apply(lambda x: eval(x))
data=data[data['no_punc'].notnull()].reset_index().drop('index', axis=1).drop(['tokes', 'datetime'], axis=1)
data.head()

/tmp/ipykernel_799/1253446902.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/mnt/c/Users/LiuD/OneDrive - University of Arkansas for Medical Sciences/Grad School/FA2022 - Project Rotation/Code/Data/no_punc_tokes.csv', index_col='Unnamed: 0')


,lang,rts,likes,quotes,replies,hr,date,no_punc
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ..."
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do..."
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s..."
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ..."
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ..."


Defining which tokens to remove (mostly stopwords) and which to keep

In [3]:
bad = ["'", 't', 'you', 'i', 'it', 's', 'don', 'they', 'that', 'to', 'the', 'he', 'she', 'and', 'a', 'doesn', 'didn', 'if', 'but', '...', 'is', 'what', 'people', 'just', 'we', 'do', 'him', 'her', 'his', 'hers', 'of', 'is', '', ' ', 'people', 'are', 'for', 'a', '&', 'by', 'would', 'been', 'were', 'if', 'for', '"', '-', 'in', 'to', 'on', 'as', 'was', 'my', 'had', 'can', 'do', 'so', 'are', '%', '/', 'by', 'in', 'it', 'to', '|', "'", 'via', 'says', '*', 'of', ' ', 'that', 'from' "she's", '[', ']','i', 'a', 'you', 'he', 'it', 'they', 'take', 'is', 'for', 'of', 'on', 'but', 'this', 'my', 'your', 'yours', 'mine', 'me', 'be', 'like', 'have', 'some', 'had', 'with', 'here', 'them', 'there', 'without', 'be', 'm', 'has', '’']
good = ['who','cdc', 'fda','ivermectin', 'horse', 'covid']

## Using Optuna to optimize the dictionary, tfidf vectorizer, and lda model simultaneously

I explored the other arguments for LDAmulticore, but eta, alpha, and decay would frequently cause ValueErrors.  While with Optuna, this doesn't reset the study, it did lead to a lot of stops and I ultimately removed them so it could train continuously without supervision.

In [18]:
def objective(trial):
    dct_no_below = trial.suggest_int('no_below', 800, 1200, step=50)
    dct_no_above = trial.suggest_float('no_above', 0.6, 0.8, step=0.05)
    dct_keep_n = trial.suggest_int('keep_n',50000, 100000, step=5000)
    #dct_bad = trial.suggest_categorical('bad_ids', [None, bad])
    #dct_good = trial.suggest_categorical('good_ids', [None, good])
    lda_num_topics = trial.suggest_int('num_topics', 10, 15, step=1)
    #lda_iterations = trial.suggest_int('iterations', 0, 10)
    #lda_eta = trial.suggest_categorical('eta',['symmetrical', 'auto'])
    #lda_passes = trial.suggest_int('passes', 0, 10, step=2)
    #lda_alpha = trial.suggest_categorical('alpha',['symmetrical', 'asymmetrical'])
    #lda_decay = trial.suggest_categorical('decay', [None, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    lda_minimum_probability = trial.suggest_float('minimum_probability', 0.2, 0.5, step=0.05)
    lda_per_word_topics = trial.suggest_categorical('per_word_topics',[True,False])

    dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
    corpus = [dct.doc2bow(line) for line in data.no_punc]

    gt=[]
    for k in good:
        try:
            gt.append(dct.token2id[k])
        except:
            continue

    dct.filter_extremes(
        no_below=dct_no_below, 
        no_above=dct_no_above, 
        keep_n=dct_keep_n,
        keep_tokens=gt)

    bt=[]
    for j in list(set(bad)):
        try:
            bt.append(dct.token2id[j])
        except:
            continue

    dct.filter_tokens(
        bad_ids=bt
    )
    corpus = [dct.doc2bow(doc) for doc in data.no_punc]
    model = TfidfModel(corpus)
    lda = LdaMulticore(model[corpus], id2word=dct, workers=9, 
        num_topics=lda_num_topics, 
        #iterations=lda_iterations, 
        #alpha = 'symmetrical',  #ValueErrors when alpha is asymmetrical, or symmetrical, can't find alpha
        #decay = lda_decay,
        random_state=19,
        minimum_probability= lda_minimum_probability,
        per_word_topics = lda_per_word_topics,
        #eta=lda_eta
        )

    cm = CoherenceModel(model=lda, texts=data.no_punc, dictionary = dct, coherence='c_v')
    coherence = cm.get_coherence()
    
    return coherence

In [6]:
study = optuna.create_study(direction = 'maximize')

[I 2022-11-25 21:42:58,382] A new study created in memory with name: no-name-9a29fa0b-38fc-4b17-812e-1232f5f2f400


In [19]:
study.optimize(objective, n_trials=18, show_progress_bar=True, gc_after_trial=True)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/18 [00:00<?, ?it/s]

[I 2022-11-26 21:55:58,378] Trial 61 finished with value: 0.42997039604023946 and parameters: {'no_below': 1050, 'no_above': 0.8, 'keep_n': 65000, 'num_topics': 13, 'minimum_probability': 0.45, 'per_word_topics': True}. Best is trial 47 with value: 0.45448753704144557.
[I 2022-11-26 22:06:22,068] Trial 62 finished with value: 0.4373857758588055 and parameters: {'no_below': 1050, 'no_above': 0.8, 'keep_n': 60000, 'num_topics': 13, 'minimum_probability': 0.35000000000000003, 'per_word_topics': True}. Best is trial 47 with value: 0.45448753704144557.
[I 2022-11-26 22:17:05,512] Trial 63 finished with value: 0.4295993458925811 and parameters: {'no_below': 1050, 'no_above': 0.8, 'keep_n': 75000, 'num_topics': 12, 'minimum_probability': 0.45, 'per_word_topics': True}. Best is trial 47 with value: 0.45448753704144557.
[I 2022-11-26 22:27:19,804] Trial 64 finished with value: 0.4452792942572674 and parameters: {'no_below': 1050, 'no_above': 0.7, 'keep_n': 65000, 'num_topics': 13, 'minimum_prob

Study best parameters after different numbers of trials

In [9]:
study.best_params  # E.g. {'x': 2.002108042}

{'no_below': 900,
 'no_above': 0.65,
 'keep_n': 80000,
 'num_topics': 14,
 'minimum_probability': 0.30000000000000004,
 'per_word_topics': False}

In [11]:
study.best_params  # E.g. {'x': 2.002108042}

{'no_below': 1000,
 'no_above': 0.7,
 'keep_n': 55000,
 'num_topics': 12,
 'minimum_probability': 0.4,
 'per_word_topics': False}

In [20]:
study.best_params  # E.g. {'x': 2.002108042}

{'no_below': 1100,
 'no_above': 0.7,
 'keep_n': 70000,
 'num_topics': 13,
 'minimum_probability': 0.45,
 'per_word_topics': True}

Running a model with the final best parameters
```
{'no_below': 1100,
 'no_above': 0.7,
 'keep_n': 70000,
 'num_topics': 13,
 'minimum_probability': 0.45,
 'per_word_topics': True}
```

In [4]:
dct = Dictionary(data.no_punc)  # fit dictionary data.tokes
corpus = [dct.doc2bow(line) for line in data.no_punc] 


gt=[]
for k in good:
    try:
        gt.append(dct.token2id[k])
    except:
        continue

dct.filter_extremes(
    no_below=1100, 
    no_above=0.7, 
    keep_n=70000,
    keep_tokens=gt)

bt=[]
for j in list(set(bad)):
    try:
        bt.append(dct.token2id[j])
    except:
        continue

dct.filter_tokens(
    bad_ids=bt
    )
corpus = [dct.doc2bow(doc) for doc in data.no_punc]
model = TfidfModel(corpus)
lda = LdaMulticore(model[corpus], id2word=dct, workers=9, 
    num_topics=13, 
    #iterations=0, 
    #alpha = lda_alpha,  #ValueErrors when alpha is asymmetrical, or symmetrical, can't find alpha
    #decay = lda_decay,
    minimum_probability= 0.45,
    random_state=19,
    per_word_topics = True
    #eta=lda_eta
    )
cm = CoherenceModel(model=lda, texts=data.no_punc, dictionary = dct, coherence='c_v')
coherence = cm.get_coherence()

In [5]:
coherence

0.43963710567427394

In [ ]:
from joblib import dump
dump(corpus, 'Data/13T Gensim/corpus.joblib')
dump(model, 'Data/13T Gensim/tfidf_model.joblib')
dump(lda, 'Data/13T Gensim/13t_lda_model.joblib')

# see LDA Topic Visualizations folder in github

In [7]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

vis_data = gensimvis.prepare(lda, model[corpus], dct)
pyLDAvis.save_html(vis_data, 'Data/1gram-ldavis-t13.html')
pyLDAvis.display(vis_data)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-

In [8]:
vis_data = gensimvis.prepare(lda, model[corpus], dct, mds='tsne')
pyLDAvis.save_html(vis_data, 'Data/1gram-ldavis-t13-tsne.html')
pyLDAvis.display(vis_data)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-

In [9]:
vis_data = gensimvis.prepare(lda, model[corpus], dct, mds='mmds')
pyLDAvis.save_html(vis_data, 'Data/1gram-ldavis-t13-mmds.html')
pyLDAvis.display(vis_data)

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/nlp/lib/python3.9/site-

In [14]:
lda.print_topics()

[(0,
  '0.007*"👉" + 0.007*"covid" + 0.007*"2021" + 0.006*"’" + 0.005*"treatment" + 0.005*"use" + 0.005*"not" + 0.005*"jail" + 0.004*"🤔" + 0.004*"nih"'),
 (1,
  '0.009*"india" + 0.008*"state" + 0.008*"pradesh" + 0.008*"uttar" + 0.007*"covid" + 0.005*"free" + 0.005*"now" + 0.005*"’" + 0.005*"fill" + 0.005*"still"'),
 (2,
  '0.011*"$" + 0.011*"pharma" + 0.011*"big" + 0.008*"money" + 0.008*"’" + 0.007*"pfizer" + 0.006*"make" + 0.006*"merck" + 0.005*"covid" + 0.005*"new"'),
 (3,
  '0.012*"visit" + 0.009*"easily" + 0.009*"days" + 0.009*"buy" + 0.008*"zinc" + 0.008*"get" + 0.007*"anyone" + 0.007*"covid" + 0.006*"c" + 0.006*"d"'),
 (4,
  '0.013*"used" + 0.012*"humans" + 0.010*"not" + 0.010*"drug" + 0.010*"parasites" + 0.010*"an" + 0.009*"human" + 0.009*"cancer" + 0.009*"treat" + 0.008*"covid"'),
 (5,
  '0.142*"meta" + 0.139*"analysis" + 0.096*"know" + 0.068*"everything" + 0.056*"need" + 0.043*"about" + 0.009*"court" + 0.007*"covid" + 0.007*"hospital" + 0.007*"19"'),
 (6,
  '0.019*"rogan" + 0.0

In [15]:
lda.save('Data/13T Gensim/lda_model.joblib')

In [17]:
lda.top_topics(corpus)

[([(0.010958718, '$'),
   (0.010607295, 'pharma'),
   (0.010581819, 'big'),
   (0.008268868, 'money'),
   (0.007980562, '’'),
   (0.007379754, 'pfizer'),
   (0.006256068, 'make'),
   (0.0059907273, 'merck'),
   (0.005351514, 'covid'),
   (0.0052908733, 'new'),
   (0.0052506123, 'no'),
   (0.0051986217, 'not'),
   (0.005183462, 'cheap'),
   (0.004993583, 'pill'),
   (0.004882597, 'drug'),
   (0.004374229, 'why'),
   (0.0043441793, 'made'),
   (0.00423412, 'effective'),
   (0.00409076, 'patent'),
   (0.0040877326, 'from')],
  -2.6625342915465713),
 ([(0.008500774, '’'),
   (0.0070435735, 'not'),
   (0.00678406, '@pierrekory'),
   (0.006490553, 'side'),
   (0.006080848, 'effects'),
   (0.0055087856, 'no'),
   (0.005442566, 'covid'),
   (0.0047924086, 'than'),
   (0.004685946, 'work'),
   (0.004317628, 'or'),
   (0.0040277396, 'more'),
   (0.0040127, 'about'),
   (0.0038088895, 'studies'),
   (0.0035564005, 'long'),
   (0.0035145748, 'at'),
   (0.003504876, 'from'),
   (0.0035000453, 'all'

In [18]:
data.head()

,lang,rts,likes,quotes,replies,hr,date,no_punc
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ..."
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do..."
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s..."
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ..."
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ..."


In [68]:
def topic_pred(tokes):
    new_doc_bow = dct.doc2bow(tokes)
    topics = lda.get_document_topics(new_doc_bow)
    if topics == []:
        return 99
    else:
        return topics[0][0]

In [71]:
data['topics']=data.no_punc.apply(topic_pred)

In [70]:
data.head()

,lang,rts,likes,quotes,replies,hr,date,no_punc,topics
0,en,0.0,1.0,0.0,1,20.0,2020-03-01,"[@ronnie_haydon, almost, certainly, mange, i, ...",7.0
1,en,0.0,0.0,0.0,0,17.0,2020-03-01,"[[, ivermectin, paste, dewormer, -, 608, g, do...",7.0
2,en,0.0,8.0,0.0,0,12.0,2020-03-01,"[🏁, the, bohemia, team, is, gearing, up, to, s...",1.0
3,en,1.0,4.0,0.0,0,15.0,2020-03-02,"[@ginagh, works, as, #isglobal, director, of, ...",11.0
4,en,0.0,0.0,0.0,1,21.0,2020-03-03,"[@liezjkc, categorically, untrue, warren, is, ...",3.0


In [72]:
topics_dates=data[['topics', 'date']]

In [73]:
topics_dates.to_csv('Data/13T Gensim/topics_dates.csv')

In [80]:
import pandas as pd
topics_dates = pd.read_csv('Data/13T Gensim/topics_dates.csv')

In [83]:
top_plot=topics_dates.groupby(['date', 'topics']).count()
top_plot.index.get_level_values('date')
import plotly.express as px
fig = px.line(topics_dates.groupby(['date', 'topics']).count(), x = top_plot.index.get_level_values('date'), y='Unnamed: 0', color=top_plot.index.get_level_values('topics'), width=1500, height=600)
fig.show()

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/root/miniconda3/envs/nlp/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

